# Collaborative Filtering

Teknik collaborative filtering ini adalah teknik yang menggunakan orang lain untuk merekomendasikan sesuatu ke input user yang memiliki preferensi yang sama. Teknik yang akan saya gunakan untuk mengukur similarity adalah pearson correlation function.

Proses dalam Collaborative Filtering kali ini adalah:

- Pilih user dengan movienya yang sudah ia tonton.
- Berdasarkan rating dari filmnya, akan mencari Top X neighbors. 
- Menghitung similarity score.
- Rekomendasikan film yang memiliki score paling tinggi

## Case Story
- Rangga merupakan seorang data scientist dari yukbeli.com pindah ke netclix.com, ia ditugaskan oleh manager barunya yang bernama Asep untuk memberikan rekomendasi film untuk anak managernya yang bernama Cinta berdasarkan film yang sudah ditonton anaknya.
- Berdasaran dataset yang diberikan, Cinta mempunyai User ID 600

## Load Library

In [2]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Load Datasets
- Dataset yang digunakan berasal dari MovieLens dengan 2 datasets yang akan kita gunakan yaitu data ratings dan movies

In [14]:
movies = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\movies.csv', encoding='latin1')
ratings = pd.read_csv('C:\\Users\\msi\\Documents\\Jupyter\\18. Unsupervised 2\\proyek\\dataset\\ratings.csv')

## Data Cleaning

In [15]:
#Menghapus kolom yang tidak digunakan di df movies
movies.drop(columns=['genres'], inplace=True)

In [16]:
#Menghapus tahun di title
movies['title'] = movies['title'].str.replace('(\(\d\d\d\d\))', '')

#Menghapus whitesoace
movies['title'] = movies['title'].apply(lambda x: x.strip())

In [17]:
#Menghapus kolom timestamp di df ratings
ratings.drop(columns=['timestamp'], inplace=True)

- Menggunakan Pearson Correlation Function:
$$
r=\frac{\sum  ( x-\bar{x} ) ( y-\bar{y} ) }{\sqrt{\sum ( x-\bar{x} )^{2} ( y-\bar{y} )^{2}}}
$$

## Modelling

In [18]:
def rekomendasi_film_cf(user_id):
    #Kita membuat movie input dari user
    rating_grouping = ratings.groupby('userId')
    user_id_film = rating_grouping.get_group(user_id)
    
    #Mencari user yang sudah melihat input movie
    list_movie = rating_grouping.get_group(1)[['movieId','rating']].sort_values(by='rating', ascending=False).head(5)
    set_user = ratings[ratings['movieId'].isin(user_id_film['movieId'].tolist())]
    set_user_grouping = set_user.groupby(['userId'])
    
    #Lalu saya akan melakukan sorting data user berdasarkan kesamaan film input dan film yang sudah ditonton user.
    set_user_grouping = sorted(set_user_grouping,  key=lambda x: len(x[1]), reverse=True)
    
    #Membuat dictionary dimana user id adalah key dan value adalah koefisien pearson
    pearson_dict = {}

    #Looping di set_user_grouping
    for name, group in set_user_grouping:
    
        #Kita sorting dulu agar tidak mixed up
        group = group.sort_values(by='movieId')
        user_id_film = user_id_film.sort_values(by='movieId')
    
        #cari jumlah dari data 
        n_value = len(group)

        #cari review scores movies yang disesuaikan dengan film yang sudah ditonton cinta
        df_sementara = user_id_film[user_id_film['movieId'].isin(group['movieId'].tolist())]

        #Simpan rating menjadi bentuk list
        rating_list = df_sementara['rating'].tolist()

        #Masukan juga reviews satu satu dari user yang ada di set_user_grouping kedalam bentuk list
        grup_list = group['rating'].tolist()

        #Lalu lakukan perhitungan untuk pearson correlation antara 2 users, cinta dan current user = x and y
        Sxx = sum([i**2 for i in rating_list]) - pow(sum(rating_list),2)/float(n_value)
        Syy = sum([i**2 for i in grup_list]) - pow(sum(grup_list),2)/float(n_value)
        Sxy = sum( i*j for i, j in zip(rating_list, grup_list)) - sum(rating_list)*sum(grup_list)/float(n_value)

        #Checking apakah penyebutnya 0 atau tidak, kalo tidak maka bisa dibagi, kalo tidak maka 0.
        if Sxx != 0 and Syy != 0:
            pearson_dict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearson_dict[name] = 0
            
    #Organisasi pearson dictionary ke dalam dataframe
    df_pearson = pd.DataFrame.from_dict(pearson_dict, orient='index')
    df_pearson.columns = ['similarityIndex']
    df_pearson['userId'] = df_pearson.index
    df_pearson.index = range(len(df_pearson))
    
    #Kita cari top 50 aja yang similarity indexnya paling mendekati dengan input user
    top_50=df_pearson.sort_values(by='similarityIndex', ascending=False)[0:50]
    
    #Tahap Rekomendasi Film
    #Pertama, merge dulu ratings dan top50
    top_50_new = top_50.merge(ratings, left_on='userId', right_on='userId', how='inner')
    
    #Kedua, Kalikan similarity index dan rating
    top_50_new['bobot_rating'] = top_50_new['similarityIndex']*top_50_new['rating']
    
    #Ketiga, groupby berdasarkan movieID dan hanya mengambil kolom similarity index dan bobot_rating
    bobot_df = top_50_new.groupby('movieId').sum()[['similarityIndex','bobot_rating']]
    bobot_df.columns = ['similarity_index_total','bobot_rating_total']
    
    #Keempat, kita bagi bobot rating total dengan similarity index total
    bobot_df['final_recommendation_score'] = bobot_df['bobot_rating_total']/bobot_df['similarity_index_total']
    bobot_df['movieId'] = bobot_df.index
    
    #Kelima, kita sorting dari nilai recommendation score yang paling tinggi
    final_rekomenadasi_film = bobot_df.sort_values(by=['final_recommendation_score'], ascending=False)
    final_rekomenadasi_film.drop(columns=['similarity_index_total', 'bobot_rating_total'], inplace=True)
    
    final_movie_rekomendasi = movies.loc[movies['movieId'].isin(final_rekomenadasi_film.head(10)['movieId'].tolist())]
    return final_movie_rekomendasi

In [19]:
user = int(input('Masukan User ID yang ingin direkomendasikan: '))
rekomendasi_film_cf(user)

Masukan User ID yang ingin direkomendasikan: 600


,movieId,title
485,553,Tombstone
1644,2193,Willow
2654,3552,Caddyshack
3087,4144,In the Mood For Love (Fa yeung nin wa)
3559,4874,K-PAX
5773,31364,Memories of Murder (Salinui chueok)
6365,49772,"Painted Veil, The"
6486,53121,Shrek the Third
6589,55276,Michael Clayton
6944,65261,Ponyo (Gake no ue no Ponyo)


- Jadi setelah melakukan analisis, Rangga melaporkan hasilnya kepada bosnya yaitu top 10 film yang wajib ditonton oleh Cinta:
    1. Tombstone
    2. Willow
    3. Caddyshack
    4. In the Mood For Love (Fa yeung nin wa)
    5. K-PAX
    6. Memories of Murder (Salinui chueok)
    7. Painted Veil, The
    8. Shrek the Third
    9. Michael Clayton
    10. Ponyo (Gake no ue no Ponyo)